Imports

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import pyodbc
import matplotlib.pyplot as plt
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression

Gather data

In [ ]:
def getDbAsDF(conn: pyodbc.Connection, sql:str):
    cursor = conn.cursor()
    cursor.execute(sql)
    data = cursor.fetchall()

    columns = [column[0] for column in cursor.description]

    formatted_data = {}

    for i in range(len(columns)):
        dataList = []
        for j in data:
            dataList.append(j[i])
        formatted_data[columns[i]] = dataList

    df = pd.DataFrame(data=formatted_data,columns=columns)
    return df


salesConn = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=..\..\Data\go_sales_schoon.accdb;')
#WHY THE FUCK DOES ACCESS REQUIRE BRACKETS FOR MORE THAN ONE JOIN AND WHY DIT IT TAKE SO LONG FOR ME TO FIND THIS INFO REEEEEEEEEEEEEEEEEEEEE
salesSQL = """
SELECT *
FROM (((((returned_item
INNER JOIN return_reason
ON returned_item.RETURN_REASON_CODE = return_reason.RETURN_REASON_CODE)
INNER JOIN order_details
ON returned_item.ORDER_DETAIL_CODE = order_details.ORDER_DETAIL_CODE)
INNER JOIN order_header
ON order_header.ORDER_NUMBER = order_details.ORDER_NUMBER)
INNER JOIN sales_sales_branch
ON sales_sales_branch.SALES_BRANCH_CODE = order_header.SALES_BRANCH_CODE)
INNER JOIN product
ON product.PRODUCT_NUMBER = order_details.PRODUCT_NUMBER)
INNER JOIN product_type
ON product_type.PRODUCT_TYPE_CODE = product.PRODUCT_TYPE_CODE
"""

salesDF = getDbAsDF(salesConn, salesSQL)
salesDF.columns

Selecting suitable data

In [ ]:
workingDF = salesDF[["RETURN_QUANTITY","RETURN_REASON_CODE","ORDER_METHOD_CODE","COUNTRY_CODE","PRODUCT_TYPE_CODE","SALES_BRANCH_CODE","UNIT_PRICE","PRODUCTION_COST","PRODUCT_NUMBER"]] # Don't know why this duplication happens 
workingDF = workingDF.T.drop_duplicates().T # But fix it here
workingDF.dropna(inplace = True)
print(workingDF.head())
print(workingDF.columns)

Creating correlation heatmap

In [ ]:
corr_matrix = workingDF.corr()
sns.heatmap(corr_matrix, annot=True)
plt.show()

In [ ]:
sns.scatterplot(x='RETURN_REASON_CODE',
                y='RETURN_QUANTITY', data=workingDF)

In [ ]:
X = workingDF.drop('RETURN_QUANTITY',axis= 1)
y = workingDF['RETURN_QUANTITY']
print(X)
print(y)

In [ ]:
# creating train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=101)

In [ ]:
# creating a regression model
model = LinearRegression()

# fitting the model
model.fit(X_train,y_train)

# making predictions
predictions = model.predict(X_test)


In [ ]:
# model evaluation

print(
  'mean_squared_error : ', mean_squared_error(y_test, predictions))
print(
  'mean_absolute_error : ', mean_absolute_error(y_test, predictions))